# Belegungsquote der Frauenhäuser in Deutschland

## Load source data and format

In [1]:
import pandas as pd
import numpy as np

fpath = "frauenhaus_suche.json"

# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

# get keys
keys = pd.read_csv("data/helpers/free_places_key.csv")

# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])
    
# reformat timeseries
df.timestamp = pd.to_datetime(df.timestamp, dayfirst=True)
df["date"] = df.timestamp.dt.date

# get monthyear as column
df["monthyear"] = df.timestamp.dt.to_period("M")

# add text keys
df.loc[df.freePlaces=="", "freePlaces"] = np.nan
df.freePlaces = df.freePlaces.astype(float)
df = pd.merge(df, keys, on="freePlaces")

# fill nas
df.loc[df.description.isnull(), "description"] = "k.A."

# add simplified NAs
desc_short = {
    "Aufnahme möglich für Frauen mit 4 oder mehr Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen mit 3 Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen mit 2 Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen ohne Kinder, Aufnahme möglich für Frauen mit 1 Kind":"Aufnahme möglich",
    "Aufnahme möglich für Frauen ohne Kinder":"Aufnahme möglich",
    "Keine Aufnahme möglich":"Keine Aufnahme möglich",
    "Aufnahme möglich ohne detaillierte Angabe":"Aufnahme möglich",
    "k.A.":"keine Angabe"
}

df["status"] = df.description.map(desc_short)

In [2]:
# trim based on dates
df = df.loc[(df.timestamp >= "2022-01-01") & (df.timestamp < "2023-07-01"),]

## Calculate overall counts of days/timestamps to use as denominator

In [3]:
# calculate counts for dividing
n_timestamps = len(df.timestamp.unique())
n_days = len(df.date.unique())
n_months = len(df.monthyear.unique())

## Load in geodata and metadata

In [4]:
# add bundesland
geocoded = pd.read_csv("data/helpers/shelters_geocoded.csv").drop(["latitude","longitude"], axis=1)
geocoded["kreis"] = geocoded.apply(lambda row : f"{row.bez} {row.gen}", axis=1)
geocoded = geocoded[["shelter_id","bundesland","kreis"]]

In [5]:
# load metadata
metadata = pd.read_csv("data/helpers/shelters_metadata.csv")
metadata = metadata.drop(['title'], axis=1)

## Start list of Frauenhäuser that don't meet data standards
First step: filter out shelters that showed up in the data less than 80% of the time

In [6]:
data_completeness = df.groupby(["shelter_id"]).timestamp.count().reset_index()
data_completeness["pct_data_availability"] = data_completeness["timestamp"]/n_timestamps
to_drop = data_completeness.loc[data_completeness.pct_data_availability < .8, "shelter_id"].values

Second step: get all shelters with more than 20% keine Angabe (based on raw data, not on daily summary)

In [7]:
status_counts = df.groupby(["shelter_id", "status"]).agg(
    status_count = ("status","count")
).reset_index()
status_counts = status_counts.loc[status_counts.status=="keine Angabe",]
status_counts["pct"] = status_counts["status_count"]/n_timestamps
to_add = status_counts.loc[status_counts.pct > .20, "shelter_id"].values
to_drop = np.concatenate([to_drop, to_add])

Third step: Filter out entries that arent Frauenhäuser

In [8]:
to_add = metadata.loc[metadata.einrichtungsart != "Frauenhaus", "shelter_id"].values
to_drop = np.concatenate([to_drop, to_add])

Map to source data

In [9]:
df["to_include"] = df.shelter_id.map(lambda x : x not in to_drop)

## Reshape data as one value per day
3x keine Angabe = keine Angabe, mind. 1 aufnahme möglich = aufnahme möglich, sonst aufnahme nicht möglich

In [10]:
# get counts per day
df_long = df.loc[df.to_include==True,] \
    .groupby(["shelter_name","shelter_id","date","status"]).agg(
        status_count = ("status", "count")
    ).reset_index()

df_long.head()

,shelter_name,shelter_id,date,status,status_count
0,1. Autonomes Frauenhaus,2027,2022-01-01,Keine Aufnahme möglich,3
1,1. Autonomes Frauenhaus,2027,2022-01-02,Keine Aufnahme möglich,3
2,1. Autonomes Frauenhaus,2027,2022-01-03,Keine Aufnahme möglich,3
3,1. Autonomes Frauenhaus,2027,2022-01-04,Keine Aufnahme möglich,3
4,1. Autonomes Frauenhaus,2027,2022-01-05,Keine Aufnahme möglich,3


In [11]:
# long to wide
df_daily = pd.pivot(
    df_long,
    index=["shelter_name","shelter_id","date"],
    values="status_count",
    columns="status"
).reset_index().replace(np.nan, 0)

# calculate counts on day, sometimes scraper failed
df_daily["day_counts"] = df_daily["Aufnahme möglich"] + df_daily["Keine Aufnahme möglich"] + df_daily["keine Angabe"]

df_daily["status"] = "keine Aufnahme möglich"
df_daily.loc[df_daily["keine Angabe"] == df_daily["day_counts"], "status"] = "keine Angabe"
df_daily.loc[df_daily["Aufnahme möglich"] > 0, "status"] = "Aufnahme möglich"

df_daily.head()

status,shelter_name,shelter_id,date,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,day_counts,status
0,1. Autonomes Frauenhaus,2027,2022-01-01,0.0,3.0,0.0,3.0,keine Aufnahme möglich
1,1. Autonomes Frauenhaus,2027,2022-01-02,0.0,3.0,0.0,3.0,keine Aufnahme möglich
2,1. Autonomes Frauenhaus,2027,2022-01-03,0.0,3.0,0.0,3.0,keine Aufnahme möglich
3,1. Autonomes Frauenhaus,2027,2022-01-04,0.0,3.0,0.0,3.0,keine Aufnahme möglich
4,1. Autonomes Frauenhaus,2027,2022-01-05,0.0,3.0,0.0,3.0,keine Aufnahme möglich


## See overall shelter counts by bundesland

In [12]:
# calculate overall number of shelters we can use by bundesland
df = pd.merge(df, geocoded, on="shelter_id")
shelter_counts = df[["shelter_id","bundesland"]].drop_duplicates().groupby(["bundesland"]).agg(
    n_shelters = ("bundesland", "count")
).reset_index()
shelter_included = df.loc[df.to_include==True,["shelter_id","bundesland"]].drop_duplicates().groupby(["bundesland"]).agg(
    n_analyzed = ("bundesland", "count")
).reset_index()
shelter_counts = pd.merge(shelter_counts, shelter_included, how="outer").replace(np.nan, 0)
shelter_counts

,bundesland,n_shelters,n_analyzed
0,Baden-Württemberg,50,23.0
1,Bayern,42,18.0
2,Berlin,12,0.0
3,Brandenburg,10,3.0
4,Bremen,3,0.0
5,Hamburg,5,0.0
6,Hessen,27,27.0
7,Mecklenburg-Vorpommern,10,4.0
8,Niedersachsen,29,9.0
9,Nordrhein-Westfalen,63,62.0


## Overall summary entire timeframe

In [15]:
cts = df_daily.groupby("status").agg(
    n = ("status", "count")
).reset_index()
cts["pct"] = round(100 * cts["n"]/cts["n"].sum(), 2)
cts.drop(["n"], axis=1)

,status,pct
0,Aufnahme möglich,13.92
1,keine Angabe,1.22
2,keine Aufnahme möglich,84.86


## Overall shelter-level summary

In [13]:
df_shelter = df_daily.groupby(["shelter_name","shelter_id","status"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter["pct"] = df_shelter["n"]/n_days
df_shelter

,shelter_name,shelter_id,status,n,pct
0,1. Autonomes Frauenhaus,2027,Aufnahme möglich,315,0.576923
1,1. Autonomes Frauenhaus,2027,keine Aufnahme möglich,231,0.423077
2,1. Frauenhaus Köln,2140,Aufnahme möglich,17,0.031136
3,1. Frauenhaus Köln,2140,keine Aufnahme möglich,529,0.968864
4,2. Autonomes Frauenhaus Köln,2253,Aufnahme möglich,27,0.049451
...,...,...,...,...,...
403,Städtisches Frauenhaus Stuttgart,2197,Aufnahme möglich,158,0.289377
404,Städtisches Frauenhaus Stuttgart,2197,keine Angabe,67,0.122711
405,Städtisches Frauenhaus Stuttgart,2197,keine Aufnahme möglich,321,0.587912
406,Wolfsburger Frauenhaus,2383,Aufnahme möglich,145,0.265568


In [14]:
# pivot to wide and fill values
df_shelter_wide = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id"],
    columns="status",
    values="pct"
).reset_index().replace(np.nan, 0)

# drop bad ids
df_shelter_wide = df_shelter_wide.loc[~df_shelter_wide.shelter_id.isin(to_drop),]

# and add geocoding
df_shelter_wide = pd.merge(df_shelter_wide, geocoded, how="left")
df_shelter_wide.head()

,shelter_name,shelter_id,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,bundesland,kreis
0,1. Autonomes Frauenhaus,2027,0.576923,0.0,0.423077,Sachsen,Kreisfreie Stadt Leipzig
1,1. Frauenhaus Köln,2140,0.031136,0.0,0.968864,Nordrhein-Westfalen,Kreisfreie Stadt Köln
2,2. Autonomes Frauenhaus Köln,2253,0.049451,0.0,0.950549,Nordrhein-Westfalen,Kreisfreie Stadt Köln
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,0.252747,0.0,0.747253,Sachsen,Kreisfreie Stadt Leipzig
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,0.009158,0.0,0.990842,Baden-Württemberg,Kreis Bodenseekreis


In [15]:
# drop shelter id
df_shelter_wide = df_shelter_wide.drop(["shelter_id"], axis=1)

In [16]:
# filter out shelters data and save to csv
df_shelter_wide.to_csv("./data/cleaned/data_by_shelter.csv", index=False)

## Shelter-level data by month

In [17]:
# get monthyear
df_daily["monthyear"] = pd.to_datetime(df_daily.date).dt.to_period("M")

In [18]:
# get count of values per day
df_monthyear = df_daily.groupby(["shelter_name","shelter_id","monthyear","status"]).agg(
    status_count = ("status", "count")
).reset_index()
df_monthyear = df_monthyear.loc[~df_monthyear.shelter_id.isin(to_drop),] # filter out incomplete, dont need to share with lokalen
df_monthyear.head()

,shelter_name,shelter_id,monthyear,status,status_count
0,1. Autonomes Frauenhaus,2027,2022-01,Aufnahme möglich,1
1,1. Autonomes Frauenhaus,2027,2022-01,keine Aufnahme möglich,30
2,1. Autonomes Frauenhaus,2027,2022-02,Aufnahme möglich,28
3,1. Autonomes Frauenhaus,2027,2022-03,Aufnahme möglich,31
4,1. Autonomes Frauenhaus,2027,2022-04,Aufnahme möglich,30


In [19]:
# pivot to wide and fill values
df_monthyear_wide = pd.pivot(
    df_monthyear,
    index=["shelter_name","shelter_id","monthyear"],
    columns="status",
    values="status_count"
).reset_index().replace(np.nan, 0)
df_monthyear_wide.head()

status,shelter_name,shelter_id,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich
0,1. Autonomes Frauenhaus,2027,2022-01,1.0,0.0,30.0
1,1. Autonomes Frauenhaus,2027,2022-02,28.0,0.0,0.0
2,1. Autonomes Frauenhaus,2027,2022-03,31.0,0.0,0.0
3,1. Autonomes Frauenhaus,2027,2022-04,30.0,0.0,0.0
4,1. Autonomes Frauenhaus,2027,2022-05,31.0,0.0,0.0


In [20]:
# get days in each month
day_counts = df_daily[["monthyear","date"]].drop_duplicates().groupby(["monthyear"]).agg(
    timestamp_count = ("date", "count")
).reset_index()

In [21]:
# add total count of timestamp for percent denominator
df_monthyear_wide = pd.merge(df_monthyear_wide, day_counts, how="outer")
df_monthyear_wide.head()

,shelter_name,shelter_id,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,timestamp_count
0,1. Autonomes Frauenhaus,2027,2022-01,1.0,0.0,30.0,31
1,1. Frauenhaus Köln,2140,2022-01,4.0,0.0,27.0,31
2,2. Autonomes Frauenhaus Köln,2253,2022-01,4.0,0.0,27.0,31
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,2022-01,31.0,0.0,0.0,31
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,2022-01,0.0,0.0,31.0,31


In [22]:
# calculate percentages
df_monthyear_wide.iloc[:,-4:-1] = df_monthyear_wide.iloc[:,-4:-1].apply(lambda x : x / df_monthyear_wide.timestamp_count, axis=0)
df_monthyear_wide.head()

,shelter_name,shelter_id,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,timestamp_count
0,1. Autonomes Frauenhaus,2027,2022-01,0.032258,0.0,0.967742,31
1,1. Frauenhaus Köln,2140,2022-01,0.129032,0.0,0.870968,31
2,2. Autonomes Frauenhaus Köln,2253,2022-01,0.129032,0.0,0.870968,31
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,2022-01,1.000000,0.0,0.000000,31
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,2022-01,0.000000,0.0,1.000000,31


In [23]:
# merge with geodata
df_monthyear_wide = pd.merge(df_monthyear_wide, geocoded, on="shelter_id", how="left")

In [24]:
# drop timestamp camp and save file
df_monthyear_wide = df_monthyear_wide.drop(["timestamp_count", "shelter_id"], axis=1)

In [25]:
df_monthyear_wide.to_csv("./data/cleaned/data_by_month_and_shelter.csv", index=False)

## Bundesland-level summary

In [26]:
df_daily_geocoded = pd.merge(df_daily, geocoded)

In [27]:
bundesland_counts = df_daily_geocoded.groupby(["bundesland"]).agg(
    n = ("bundesland", "count")
).reset_index()

In [28]:
bundesland = df_daily_geocoded.groupby(["bundesland", "status"]).agg(
    status_count = ("status", "count")
).reset_index()

bundesland = pd.merge(bundesland, bundesland_counts, on="bundesland")
bundesland["pct"] = bundesland["status_count"]/bundesland["n"]

bundesland_wide = pd.pivot(bundesland, index="bundesland", columns="status", values="pct").reset_index()

bundesland_wide = pd.merge(bundesland_wide, shelter_counts, on="bundesland", how="outer").replace(np.nan, 0)

bundesland_wide.head()

,bundesland,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,n_shelters,n_analyzed
0,Baden-Württemberg,0.120035,0.035215,0.844750,50,23.0
1,Bayern,0.160053,0.018620,0.821327,42,18.0
2,Brandenburg,0.115385,0.000000,0.884615,10,3.0
3,Hessen,0.084600,0.000000,0.915400,27,27.0
4,Mecklenburg-Vorpommern,0.160256,0.086081,0.753663,10,4.0


In [29]:
bundesland_wide.to_csv("./data/cleaned/bundesland_overview.csv", index=False)

## Bundesland and month-level summary

In [30]:
bundesland_counts = df_daily_geocoded.groupby(["bundesland","monthyear"]).agg(
    n = ("bundesland", "count")
).reset_index()

In [31]:
bundesland = df_daily_geocoded.groupby(["bundesland", "monthyear", "status"]).agg(
    status_count = ("status", "count")
).reset_index()

bundesland = pd.merge(bundesland, bundesland_counts, on=["bundesland","monthyear"])
bundesland["pct"] = bundesland["status_count"]/bundesland["n"]

bundesland_wide = pd.pivot(bundesland, index=["bundesland","monthyear"], columns="status", values="pct").reset_index()

bundesland_wide = pd.merge(bundesland_wide, shelter_counts, on="bundesland", how="outer").replace(np.nan, 0)

bundesland_wide.head()

,bundesland,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,n_shelters,n_analyzed
0,Baden-Württemberg,2022-01,0.268328,0.080645,0.651026,50,23.0
1,Baden-Württemberg,2022-02,0.285714,0.045455,0.668831,50,23.0
2,Baden-Württemberg,2022-03,0.127507,0.050143,0.822350,50,23.0
3,Baden-Württemberg,2022-04,0.124638,0.040580,0.834783,50,23.0
4,Baden-Württemberg,2022-05,0.117812,0.033661,0.848527,50,23.0


In [32]:
bundesland_wide.to_csv("./data/cleaned/monthly_bundesland_overview.csv", index=False)

## Germany-wide month-level summary

In [33]:
monthly_counts = df_daily_geocoded.groupby(["monthyear"]).agg(
    n = ("monthyear", "count")
).reset_index()

In [34]:
monthly = df_daily_geocoded.groupby(["monthyear", "status"]).agg(
    status_count = ("status", "count")
).reset_index()

monthly = pd.merge(monthly, monthly_counts, on="monthyear")
monthly["pct"] = monthly["status_count"]/monthly["n"]

monthly_wide = pd.pivot(monthly, index=["monthyear"], columns="status", values="pct").reset_index()

monthly_wide.head()

status,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich
0,2022-01,0.266790,0.019744,0.713466
1,2022-02,0.228765,0.013209,0.758026
2,2022-03,0.210031,0.009569,0.780399
3,2022-04,0.177551,0.007313,0.815136
4,2022-05,0.179065,0.005102,0.815833


In [35]:
monthly_wide.to_csv("./data/cleaned/monthly.csv", index=False)

## Shelter-level summary with coordinates

In [36]:
df_shelter = df_daily.groupby(["shelter_name","shelter_id","status"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter["pct"] = df_shelter["n"]/n_days
df_shelter.head()

,shelter_name,shelter_id,status,n,pct
0,1. Autonomes Frauenhaus,2027,Aufnahme möglich,315,0.576923
1,1. Autonomes Frauenhaus,2027,keine Aufnahme möglich,231,0.423077
2,1. Frauenhaus Köln,2140,Aufnahme möglich,17,0.031136
3,1. Frauenhaus Köln,2140,keine Aufnahme möglich,529,0.968864
4,2. Autonomes Frauenhaus Köln,2253,Aufnahme möglich,27,0.049451


In [37]:
# pivot to wide and fill values
df_shelter_wide = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id"],
    columns="status",
    values="pct"
).reset_index().replace(np.nan, 0)

# drop bad ids
df_shelter_wide = df_shelter_wide.loc[~df_shelter_wide.shelter_id.isin(to_drop),]

# and add geocoding
df_shelter_wide = pd.merge(df_shelter_wide, geocoded, how="left")
df_shelter_wide.head()

,shelter_name,shelter_id,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,bundesland,kreis
0,1. Autonomes Frauenhaus,2027,0.576923,0.0,0.423077,Sachsen,Kreisfreie Stadt Leipzig
1,1. Frauenhaus Köln,2140,0.031136,0.0,0.968864,Nordrhein-Westfalen,Kreisfreie Stadt Köln
2,2. Autonomes Frauenhaus Köln,2253,0.049451,0.0,0.950549,Nordrhein-Westfalen,Kreisfreie Stadt Köln
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,0.252747,0.0,0.747253,Sachsen,Kreisfreie Stadt Leipzig
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,0.009158,0.0,0.990842,Baden-Württemberg,Kreis Bodenseekreis


In [38]:
dropped = df.loc[df.to_include==False, ["shelter_name","shelter_id"]].drop_duplicates()
dropped = pd.merge(dropped, geocoded, how="left")
dropped.head()

,shelter_name,shelter_id,bundesland,kreis
0,Hestia-Zufluchtswohnungen Wedding,2051,Berlin,Kreisfreie Stadt Berlin
1,offensiv´91 e.V. Frauenzufluchtswohnung,2037,Berlin,Kreisfreie Stadt Berlin
2,Frauenhaus Bitterfeld-Wolfen,2295,Sachsen-Anhalt,Landkreis Anhalt-Bitterfeld
3,Frauenschutzangebot,2210,Sachsen,Landkreis Erzgebirgskreis
4,Frauenhäuser Geschütztes Wohnen,2327,Baden-Württemberg,Landkreis Karlsruhe


In [39]:
df_shelter_wide = pd.concat([df_shelter_wide, dropped])

In [40]:
coords = df[["shelter_id","latitude","longitude"]].drop_duplicates()

In [41]:
df_shelter_wide = pd.merge(df_shelter_wide, coords)

In [42]:
df_shelter_wide.to_csv("./data/cleaned/shelter_summary_with_coordinates.csv", index=False)

## Weihnachts Beispiel - 12.25.2022 um 8 Uhr

In [43]:
# filter by data and get relevant variables
weihnacht = df.loc[df.timestamp==pd.to_datetime("2022-25-12 08:01:00", dayfirst=True),] # filter by day
weihnacht = weihnacht[["shelter_name","shelter_id","latitude","longitude","timestamp","description","status"]]

In [44]:
# add metadata
weihnacht = pd.merge(weihnacht, metadata, on='shelter_id')

In [45]:
weihnacht.to_csv("./data/cleaned/belegungsstatus_25-12-2022_8-01.csv", index=False)